In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score,roc_curve
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/My Drive/new_data.csv")

Mounted at /content/drive


In [ ]:
X = df.drop('EverDelinquent', axis=1)
y = df['EverDelinquent'].values

smt = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_sm,y_train_sm = smt.fit_resample(X_train,y_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Logistic Regression - Chenqi

In [ ]:
##before rebalancing
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report= classification_report(y_test, y_pred)


print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

Accuracy: 75.26%
Confusion Matrix:
[[30546    55]
 [10013    76]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.75      1.00      0.86     30601
         1.0       0.58      0.01      0.01     10089

    accuracy                           0.75     40690
   macro avg       0.67      0.50      0.44     40690
weighted avg       0.71      0.75      0.65     40690



In [ ]:
## after
model_balanced = LogisticRegression(class_weight='balanced')
model_balanced.fit(X_train_scaled, y_train)

y_pred_balanced = model_balanced.predict(X_test_scaled)

accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
conf_matrix_balanced = confusion_matrix(y_test, y_pred_balanced)
report_balanced = classification_report(y_test, y_pred_balanced)

In [ ]:
print(f"Accuracy: {accuracy_balanced * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix_balanced)
print("\nClassification Report:")
print(report_balanced)

Accuracy: 60.29%
Confusion Matrix:
[[18265 12336]
 [ 3824  6265]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.60      0.69     30601
         1.0       0.34      0.62      0.44     10089

    accuracy                           0.60     40690
   macro avg       0.58      0.61      0.57     40690
weighted avg       0.71      0.60      0.63     40690



# SVM - Leyi

In [ ]:
model=LinearSVC()
model.fit(X_train_scaled, y_train)
train_pred_svc=model.predict(X_train_scaled)
y_preds_svc=model.predict(X_test_scaled)

print(f"Training score = {metrics.accuracy_score(y_train,train_pred_svc)}")
print(f"Testing score = {metrics.accuracy_score(y_test,y_preds_svc)}")

Training score = 0.7526726562253141
Testing score = 0.7520275251904645


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
training_accuracy_svc = accuracy_score(y_train,train_pred_svc)*100
print('training data accuracy is:', training_accuracy_svc)
print()

clf_report_svc = classification_report(y_train,train_pred_svc)
print('Classification report:\n', clf_report_svc)
print()

confusion_matrix(y_train,train_pred_svc)

training data accuracy is: 75.26831888606849

Classification report:
               precision    recall  f1-score   support

         0.0       0.75      1.00      0.86     71461
         1.0       0.53      0.00      0.00     23482

    accuracy                           0.75     94943
   macro avg       0.64      0.50      0.43     94943
weighted avg       0.70      0.75      0.65     94943




array([[71454,     7],
       [23474,     8]])

In [ ]:
testing_accuracy_svc = accuracy_score(y_test,y_preds_svc)*100
print('training data accuracy is:', testing_accuracy_svc)
print()

clf_report_svc = classification_report(y_test,y_preds_svc)
print('Classification report:\n', clf_report_svc)
print()

confusion_matrix(y_test,y_preds_svc)

training data accuracy is: 75.20275251904644

Classification report:
               precision    recall  f1-score   support

         0.0       0.75      1.00      0.86     30601
         1.0       0.33      0.00      0.00     10089

    accuracy                           0.75     40690
   macro avg       0.54      0.50      0.43     40690
weighted avg       0.65      0.75      0.65     40690




array([[30599,     2],
       [10088,     1]])

# LGBM - Leyi

In [ ]:
params = {
    'num_leaves': 15,
    'max_depth': 4,
    'min_data_in_leaf': 20
}
lgbm_model = lgb.LGBMClassifier(**params)
lgbm_model.fit(X_train_scaled, y_train)
train_pred_lgbm = lgbm_model.predict(X_train_scaled)
test_pred_lgbm = lgbm_model.predict(X_test_scaled)

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Info] Number of positive: 23482, number of negative: 71461
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022905 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 408
[LightGBM] [Info] Number of data points in the train set: 94943, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.247327 -> initscore=-1.112918
[LightGBM] [Info] Start training from score -1.112918
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

In [ ]:
testing_accuracy_lgbm = accuracy_score(y_test,test_pred_lgbm)*100
print('training data accuracy is:', testing_accuracy_lgbm)
print()

clf_report_lgbm = classification_report(y_test,test_pred_lgbm)
print('Classification report:\n', clf_report_lgbm)
print()

confusion_matrix(y_test,test_pred_lgbm)

training data accuracy is: 75.23470140083559

Classification report:
               precision    recall  f1-score   support

         0.0       0.76      0.99      0.86     30601
         1.0       0.51      0.02      0.04     10089

    accuracy                           0.75     40690
   macro avg       0.63      0.51      0.45     40690
weighted avg       0.70      0.75      0.66     40690




array([[30377,   224],
       [ 9853,   236]])

In [ ]:
training_accuracy_lgbm = accuracy_score(y_train,train_pred_lgbm)*100
print('training data accuracy is:', training_accuracy_lgbm)
print()

clf_report_lgbm = classification_report(y_train,train_pred_lgbm)
print('Classification report:\n', clf_report_lgbm)
print()

confusion_matrix(y_train,train_pred_lgbm)

training data accuracy is: 75.47686506640827

Classification report:
               precision    recall  f1-score   support

         0.0       0.76      0.99      0.86     71461
         1.0       0.60      0.03      0.05     23482

    accuracy                           0.75     94943
   macro avg       0.68      0.51      0.45     94943
weighted avg       0.72      0.75      0.66     94943




array([[71048,   413],
       [22870,   612]])

# Neural Network - Hsuan

**Not scaled and rebalanced:**

In [ ]:
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(6,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)
model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5)

1272/1272 [==============================] - 2s 1ms/step


**Only scaled:**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=100, batch_size=16, verbose=0)

In [ ]:
evaluation = model.evaluate(X_test_scaled, y_test)
print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])

y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5)

y_train_pred = model.predict(X_train_scaled)
y_train_pred_binary = (y_train_pred > 0.5)

training_accuracy_nn = accuracy_score(y_train, y_train_pred_binary) * 100
print('Training data accuracy is:', training_accuracy_nn)
print()

clf_report_nn = classification_report(y_test, y_pred_binary)
print('Classification report:\n', clf_report_nn)
print()

confusion_matrix_nn = confusion_matrix(y_test, y_pred_binary)
print('Confusion Matrix:\n', confusion_matrix_nn)

1272/1272 [==============================] - 5s 4ms/step - loss: 0.5356 - accuracy: 0.7522
Evaluation Loss: 0.5355596542358398
Evaluation Accuracy: 0.7521504163742065
2967/2967 [==============================] - 6s 2ms/step
Training data accuracy is: 75.33256796182974

Classification report:
               precision    recall  f1-score   support

         0.0       0.76      0.99      0.86     30601
         1.0       0.50      0.03      0.05     10089

    accuracy                           0.75     40690
   macro avg       0.63      0.51      0.45     40690
weighted avg       0.69      0.75      0.66     40690


Confusion Matrix:
 [[30346   255]
 [ 9830   259]]


**Scaled and rebalanced:**


In [ ]:
smt = SMOTE()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_sm)
X_test_scaled = scaler.transform(X_test)

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train_sm, epochs=100, batch_size=16, verbose=0)

In [ ]:
evaluation = model.evaluate(X_test_scaled, y_test)
print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy:", evaluation[1])

y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5)

y_train_pred = model.predict(X_train_scaled)
y_train_pred_binary = (y_train_pred > 0.5)

training_accuracy_nn = accuracy_score(y_train_sm, y_train_pred_binary) * 100
print('Training data accuracy is:', training_accuracy_nn)
print()

clf_report_nn = classification_report(y_test, y_pred_binary)
print('Classification report:\n', clf_report_nn)
print()

confusion_matrix_nn = confusion_matrix(y_test, y_pred_binary)
print('Confusion Matrix:\n', confusion_matrix_nn)

1272/1272 [==============================] - 2s 2ms/step - loss: 0.6638 - accuracy: 0.5945
Evaluation Loss: 0.663750171661377
Evaluation Accuracy: 0.5945441126823425
4467/4467 [==============================] - 7s 2ms/step
Training data accuracy is: 66.12557898713985

Classification report:
               precision    recall  f1-score   support

         0.0       0.82      0.59      0.69     30601
         1.0       0.33      0.61      0.43     10089

    accuracy                           0.59     40690
   macro avg       0.57      0.60      0.56     40690
weighted avg       0.70      0.59      0.62     40690


Confusion Matrix:
 [[18051 12550]
 [ 3948  6141]]


# Random Forest - Sike

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_scaled, y_train)
prediction = rf.predict(X_test_scaled)
print("For base model:")
print("The accuracy score is" , accuracy_score(y_test, prediction))
print("The F1 score is" , f1_score(y_test,prediction))
print("The auc is",roc_auc_score(y_test,prediction))

For base model:
The accuracy score is 0.6940525927746375
The F1 score is 0.2542383034805008
The auc is 0.5319311391580198


In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()
X_train_sm,y_train_sm = smt.fit_resample(X_train,y_train)
rf.fit(X_train_sm,y_train_sm)
prediction = rf.predict(X_val)
print("For base and oversampling model:")
print("The accuracy score is" , accuracy_score(y_val, prediction))
# Display F1 score
print("The F1 score is" , f1_score(y_val,prediction))
# Displat auc score
print("The auc is",roc_auc_score(y_val,prediction))

NameError: ignored

After Dealing with the imbalanced, f1 and auc increase while accuracy score decreasse. Because of regarding predict accuracy as the most important, decide not do the oversampling.

In [ ]:
#tune parameters
from sklearn.model_selection import GridSearchCV
import warnings
warnings.warn('ignore')
rf1 = RandomForestClassifier()
params = {'n_estimators':[200,400,600,800],
          'max_features':['sqrt', 'log2', "auto", 10]}
gsv_rf = GridSearchCV(rf1, params, cv=5, n_jobs=-1, scoring='accuracy')
gsv_rf.fit(X_train_scaled,y_train)

In [ ]:
gsv_rf.best_params_

{'max_features': 'log2', 'n_estimators': 800}

In [ ]:
prediction_rf = RandomForestClassifier(max_features='log2',n_estimators=800,random_state=1).fit(X_train_scaled,y_train).predict(X_test_scaled)
#
# Display accuracy score
print("The accuracy score is" , accuracy_score(y_test, prediction_rf))
# Display F1 score
print("The F1 score is" , f1_score(y_test,prediction_rf))
# Displat auc score
print("The auc is",roc_auc_score(y_test,prediction_rf))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,prediction_rf))

              precision    recall  f1-score   support

         0.0       0.77      0.86      0.81     30601
         1.0       0.32      0.21      0.25     10089

    accuracy                           0.70     40690
    macro avg      0.55      0.53      0.53     40690
    weighted avg   0.66      0.70      0.67     40690

In [ ]:
import matplotlib.pyplot as plt
#ROC curve
fpr, tpr, _ = roc_curve(y_test,prediction_rf)
#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(roc_auc_score(y_test, prediction_rf)))
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title("ROC curve for random forest")
plt.legend(loc=4)
plt.show()